# Example 02

In this example we are looking at dependencies between stages

In [1]:
from Example01 import TextToFile
from Example02 import ProcessData
from tempfile import TemporaryDirectory
import os
import shutil

In [2]:
temp_dir = TemporaryDirectory()
shutil.copy('Example01.py', temp_dir.name)
shutil.copy('Example02.py', temp_dir.name)
os.chdir(temp_dir.name)

!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmpwhzdduox/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


Content of `Example02.py`
```python
from pytrack import PyTrack
from Example01 import TextToFile


class ProcessData(PyTrack):

    def __init__(self, id_=None, filter_=None):
        super().__init__(id_, filter_)
        self.post_init(id_, filter_)

    def __call__(self):
        self.dvc.deps = [TextToFile(id_=0).files.json_file]

        self.post_call()

    def run(self):
        self.results = sum(TextToFile(id_=0).results['text'])

```

In [3]:
text_to_file = TextToFile()
process_data = ProcessData()

In [4]:
text_to_file(text=[1, 2, 3, 4])
process_data()

Contents of `dvc.yaml`

```yaml
stages:
  TextToFile_0:
    cmd: python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
    params:
    - config/params.json:
      - TextToFile.0
    outs:
    - outs\0_TextToFile.json
  ProcessData_0:
    cmd: python -c "from Example02 import ProcessData; ProcessData(id_=0).run()"
    deps:
    - outs\0_TextToFile.json
    params:
    - config/params.json:
      - ProcessData.0
    outs:
    - outs\0_ProcessData.json
```

In [5]:
!dvc repro
!git checkout -b "run_1"
!git add .
!git commit -m "run_1"

Running stage 'TextToFile_0':
> python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Running stage 'ProcessData_0':
> python -c "from Example02 import ProcessData; ProcessData(id_=0).run()"
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


Switched to a new branch 'run_1'


[run_1 (root-commit) 2e121a9] run_1
 17 files changed, 610 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/default.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/smooth.json
 create mode 100644 .dvcignore
 create mode 100644 Example01.py
 create mode 100644 Example02.py
 create mode 100644 __pycache__/Example01.cpython-39.pyc
 create mode 100644 __pycache__/Example02.cpython-39.pyc
 create mode 100644 config/params.json
 create mode 100644 dvc.lock
 create mode 100644 dvc.yaml
 create mode 100644 outs/.gitignore


In [6]:
# process_data.dvc
process_data.results

10

In [7]:
!dvc dag

+--------------+   

| TextToFile_0 |   

+--------------+   

        *          

        *          

        *          

+---------------+  

| ProcessData_0 |  

+---------------+  


In [8]:
text_to_file(text=[5, 6, 7, 8, 9])
process_data()

!dvc repro
!git checkout -b "run_2"
!git add .
!git commit -m "run_2"

ERROR: Stage 'TextToFile_0' already exists in 'dvc.yaml'. Use '--force' to overwrite.

ERROR: Stage 'ProcessData_0' already exists in 'dvc.yaml'. Use '--force' to overwrite.



Running stage 'TextToFile_0':
> python -c "from Example01 import TextToFile; TextToFile(id_=0).run()"
Updating lock file 'dvc.lock'

Running stage 'ProcessData_0':
> python -c "from Example02 import ProcessData; ProcessData(id_=0).run()"
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.


Switched to a new branch 'run_2'


[run_2 8831e84] run_2
 2 files changed, 15 insertions(+), 13 deletions(-)


In [9]:
process_data.results

35

In [10]:
!git branch
!git checkout run_1
!dvc checkout
process_data.results


  run_1
* run_2


Switched to branch 'run_1'


M       outs\0_ProcessData.json
M       outs\0_TextToFile.json


10

In [11]:
!git branch
!git checkout run_2
!dvc checkout
process_data.results

* run_1
  run_2


Switched to branch 'run_2'


M       outs\0_ProcessData.json
M       outs\0_TextToFile.json


35

In [12]:
os.chdir('..')
temp_dir.cleanup()
